# Sistema desenvolvido por Flayson Santos 


## Referência https://sefiks.com/2020/02/17/face-recognition-with-facebook-deepface-in-keras/

In [14]:
!pip install tensorflow 

In [1]:
import os
from os import listdir
import numpy as np
import cv2
from keras.models import Model, Sequential
from keras.layers import Input, Convolution2D, LocallyConnected2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from PIL import Image
from tensorflow.keras.preprocessing.image import load_img, save_img, img_to_array
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image
import matplotlib.pyplot as plt
from keras.models import model_from_json
import glob


In [2]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  # Invalid device or cannot modify virtual devices once initialized.
  pass
print("ok!")

ok!


In [3]:
target_size = (152, 152)

In [60]:
opencv_home = cv2.__file__
folders = opencv_home.split(os.path.sep)[0:-1]
path = folders[0]
for folder in folders[1:]:
	path = path + "/" + folder

detector_path = path+"/data/haarcascade_frontalface_alt.xml"

if os.path.isfile(detector_path) != True:
	raise ValueError("Confirm that opencv is installed on your environment! Expected path ",detector_path," violated.")
else:
	face_cascade = cv2.CascadeClassifier(detector_path)


In [64]:
def detectFace(img_path, target_size=(152, 152)):
	
	img = cv2.imread(img_path)
	
	faces = face_cascade.detectMultiScale(img, 1.3, 5)

	if len(faces) > 0:
		x,y,w,h = faces[0]
		
		margin = 0
		x_margin = w * margin / 100
		y_margin = h * margin / 100
		
		if y - y_margin > 0 and y+h+y_margin < img.shape[1] and x-x_margin > 0 and x+w+x_margin < img.shape[0]:
			detected_face = img[int(y-y_margin):int(y+h+y_margin), int(x-x_margin):int(x+w+x_margin)]
		else:
			detected_face = img[int(y):int(y+h), int(x):int(x+w)]
		
		detected_face = cv2.resize(detected_face, target_size)
		
		img_pixels = image.img_to_array(detected_face)
		img_pixels = np.expand_dims(img_pixels, axis = 0)
		
		#normalize in [0, 1]
		img_pixels /= 255 
		
		return img_pixels
	else:
		raise ValueError("Face could not be detected in ", img_path,". Please confirm that the picture is a face photo.")

# Transfer learning dos pesos do modelo ja treinado 

In [6]:
#DeepFace model
base_model = Sequential()
base_model.add(Convolution2D(32, (11, 11), activation='relu', name='C1', input_shape=(152, 152, 3)))
base_model.add(MaxPooling2D(pool_size=3, strides=2, padding='same', name='M2'))
base_model.add(Convolution2D(16, (9, 9), activation='relu', name='C3'))
base_model.add(LocallyConnected2D(16, (9, 9), activation='relu', name='L4'))
base_model.add(LocallyConnected2D(16, (7, 7), strides=2, activation='relu', name='L5') )
base_model.add(LocallyConnected2D(16, (5, 5), activation='relu', name='L6'))
base_model.add(Flatten(name='F0'))
base_model.add(Dense(4096, activation='relu', name='F7'))
base_model.add(Dropout(rate=0.5, name='D0'))
base_model.add(Dense(8631, activation='softmax', name='F8'))

## baixe o arquivo https://github.com/swghosh/DeepFace/releases

model.load_weights("VGGFace2_DeepFace_weights_val-0.9034.h5")

extraia e passe o caminho do diretrio em base_model.load_weights

In [7]:
base_model.load_weights("C:/Users/Flayson Santos/Dio_MachineLearn/machine_learnig/facil_detection_ofc/facial/VGGFace2_DeepFace_weights_val-0.9034.h5/VGGFace2_DeepFace_weights_val-0.9034.h5")

print ("ok loading model")

ok loading model


In [8]:
model = Model(inputs=base_model.layers[0].input, outputs=base_model.layers[-3].output)

In [9]:
def l2_normalize(x):
    return x / np.sqrt(np.sum(np.multiply(x, x)))
 
def findEuclideanDistance(source_representation, test_representation):
    euclidean_distance = source_representation - test_representation
    euclidean_distance = np.sum(np.multiply(euclidean_distance, euclidean_distance))
    euclidean_distance = np.sqrt(euclidean_distance)
    return euclidean_distance


## Base de dados para o treinamento (Voce deve criar uma pasta com arquivos .jpg e passa o caminho da pasta para employee_pictures

In [10]:
from tensorflow.keras.preprocessing import image

In [59]:
employee_pictures = "C:/Users/Flayson Santos/Dio_MachineLearn/machine_learnig/facil_detection_ofc/database"

employees = dict()

for file in listdir(employee_pictures):
    employee, extension = file.split(".")
    img_path = 'database/%s.jpg' % (employee)
    img = detectFace(img_path)
    
    representation = model.predict(img)[0]
    
    employees[employee] = representation
    
print("employee representations retrieved successfully")


1/1 [==============================] - 0s 403ms/step
employee representations retrieved successfully


# Codigo 1 parte reconhecimento facil em imagem 

In [50]:
path = "C:/Users/Flayson Santos/Dio_MachineLearn/machine_learnig/facil_detection_ofc/database/"
for infile in glob.glob(os.path.join(path, '*.jpg')):
	img = cv2.imread(infile)
	img = cv2.resize(img, (int(0.5*img.shape[1]), int(0.5*img.shape[0])), interpolation = cv2.INTER_AREA)
	faces = face_cascade.detectMultiScale(img, 1.2, 5)
	
	for (x,y,w,h) in faces:
		if w > 130: #discard small detected faces
			cv2.rectangle(img, (x,y), (x+w,y+h), (0, 255, 0), 6) #draw rectangle to main image
			
			detected_face = img[int(y):int(y+h), int(x):int(x+w)] #crop detected face
			detected_face = cv2.resize(detected_face, target_size) #resize to 152x152
			
			img_pixels = image.img_to_array(detected_face)
			img_pixels = np.expand_dims(img_pixels, axis = 0)
			img_pixels /= 255
			
			captured_representation = model.predict(img_pixels)[0]
			
			distances = []
			
			for i in employees:
				employee_name = i
				source_representation = employees[i]
				
				distance = findEuclideanDistance(l2_normalize(captured_representation), l2_normalize(source_representation))
				distances.append(distance)
				print(distance)
			is_found = False; index = 0
			for i in employees:
				employee_name = i
				if index == np.argmin(distances):
					if distances[index] <= 0.99:
						
						print("detected: ",employee_name, "(",distances[index],")")
						employee_name = employee_name.replace("_", "")
						similarity = distances[index]
						
						is_found = True
						
						break
					
				index = index + 1
			
			if is_found:
				display_img = cv2.imread("database/%s.jpg" % employee_name)
				pivot_img_size = 112
				display_img = cv2.resize(display_img, (pivot_img_size, pivot_img_size))
								
				try:
					resolution_x = img.shape[1]; resolution_y = img.shape[0]
					
					label = employee_name+" ("+"{0:.2f}".format(similarity)+")"
					
					if y - pivot_img_size > 0 and x + w + pivot_img_size < resolution_x:
						#top right
						img[y - pivot_img_size:y, x+w:x+w+pivot_img_size] = display_img
						cv2.putText(img, label, (x+w, y+10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (67,67,67), 1)					
						
						#connect face and text
						cv2.line(img,(x+int(w/2), y), (x+3*int(w/4), y-int(pivot_img_size/2)),(67,67,67),1)
						cv2.line(img, (x+3*int(w/4), y-int(pivot_img_size/2)), (x+w, y - int(pivot_img_size/2)), (67,67,67),1)
					elif y + h + pivot_img_size < resolution_y and x - pivot_img_size > 0:
						#bottom left
						img[y+h:y+h+pivot_img_size, x-pivot_img_size:x] = display_img
						cv2.putText(img, label, (x - pivot_img_size, y+h-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (67,67,67), 1)
						
						#connect face and text
						cv2.line(img,(x+int(w/2), y+h), (x+int(w/2)-int(w/4), y+h+int(pivot_img_size/2)),(67,67,67),1)
						cv2.line(img, (x+int(w/2)-int(w/4), y+h+int(pivot_img_size/2)), (x, y+h+int(pivot_img_size/2)), (67,67,67),1)
						
					elif y - pivot_img_size > 0 and x - pivot_img_size > 0:
						#top left
						img[y-pivot_img_size:y, x-pivot_img_size:x] = display_img
						cv2.putText(img, label, (x - pivot_img_size, y+10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (67,67,67), 1)
						
						#connect face and text
						cv2.line(img,(x+int(w/2), y), (x+int(w/2)-int(w/4), y-int(pivot_img_size/2)),(67,67,67),1)
						cv2.line(img, (x+int(w/2)-int(w/4), y-int(pivot_img_size/2)), (x, y - int(pivot_img_size/2)), (67,67,67),1)
						
					elif x+w+pivot_img_size < resolution_x and y + h + pivot_img_size < resolution_y:
						#bottom righ
						img[y+h:y+h+pivot_img_size, x+w:x+w+pivot_img_size] = display_img
						cv2.putText(img, label, (x+w, y+h-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (67,67,67), 1)
						
						#connect face and text
						cv2.line(img,(x+int(w/2), y+h), (x+int(w/2)+int(w/4), y+h+int(pivot_img_size/2)),(67,67,67),1)
						cv2.line(img, (x+int(w/2)+int(w/4), y+h+int(pivot_img_size/2)), (x+w, y+h+int(pivot_img_size/2)), (67,67,67),1)
					
				except Exception as e:
					print("exception occured: ", str(e))
			
	cv2.imshow('img',img)
	
	cv2.waitKey(0)
cv2.destroyAllWindows()

1/1 [==============================] - 1s 781ms/step
0.34149352
0.8434756
0.67915505
detected:  Elon Musk ( 0.34149352 )
1/1 [==============================] - 1s 741ms/step
0.7215911
0.5451348
0.7771144
detected:  Flayson Santos ( 0.5451348 )
1/1 [==============================] - 1s 799ms/step
0.7003709
0.9339839
0.4386393
detected:  linus  trovalds ( 0.4386393 )


# Codigo 2 parte reconhecimento facil em video e webcam

In [58]:
cap = cv2.VideoCapture(0) #webcam
#cap = cv2.VideoCapture("C:/Users/Flayson Santos/Dio_MachineLearn/machine_learnig/facil_detection_ofc/database/video/teste.mp4") #video
#cap = cv2.VideoCapture('digite o caminho do caminho do seu video ') #video
ret=True
while(ret==True):
    ret, img = cap.read()
    if (ret==True):
        img = cv2.resize(img, (int(0.5*img.shape[1]), int(0.5*img.shape[0])), interpolation = cv2.INTER_AREA)
        faces = face_cascade.detectMultiScale(img, 1.2, 5)
        
        for (x,y,w,h) in faces:
            if w > 130: #discard small detected faces
                cv2.rectangle(img, (x,y), (x+w,y+h), (0, 255, 0), 5) #draw rectangle to main image
                
                detected_face = img[int(y):int(y+h), int(x):int(x+w)] #crop detected face
                detected_face = cv2.resize(detected_face, target_size) #resize to 152x152
                
                img_pixels = image.img_to_array(detected_face)
                img_pixels = np.expand_dims(img_pixels, axis = 0)
                img_pixels /= 255
                
                captured_representation = model.predict(img_pixels)[0]
                
                distances = []
                
                for i in employees:
                    employee_name = i
                    source_representation = employees[i]
                    
                    distance = findEuclideanDistance(l2_normalize(captured_representation), l2_normalize(source_representation))
                    distances.append(distance)
                    print(distance)
                is_found = False; index = 0
                for i in employees:
                    employee_name = i
                    if index == np.argmin(distances):
                        if distances[index] <= 0.99:
                            
                            print("detected: ",employee_name, "(",distances[index],")")
                            #employee_name = employee_name.replace("_", "")
                            similarity = distances[index]
                            
                            is_found = True
                            
                            break
                        
                    index = index + 1
                
                if is_found:
                    display_img = cv2.imread("database/%s.jpg" % employee_name)
                    pivot_img_size = 112
                    display_img = cv2.resize(display_img, (pivot_img_size, pivot_img_size))
                                    
                    try:
                        resolution_x = img.shape[1]; resolution_y = img.shape[0]
                        
                        label = employee_name+" ("+"{0:.2f}".format(similarity)+")"
                        
                        if y - pivot_img_size > 0 and x + w + pivot_img_size < resolution_x:
                            #top right
                            img[y - pivot_img_size:y, x+w:x+w+pivot_img_size] = display_img
                            cv2.putText(img, label, (x+w, y+10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (67,67,67), 1)                    
                            
                            #connect face and text
                            cv2.line(img,(x+int(w/2), y), (x+3*int(w/4), y-int(pivot_img_size/2)),(67,67,67),1)
                            cv2.line(img, (x+3*int(w/4), y-int(pivot_img_size/2)), (x+w, y - int(pivot_img_size/2)), (67,67,67),1)
                        elif y + h + pivot_img_size < resolution_y and x - pivot_img_size > 0:
                            #bottom left
                            img[y+h:y+h+pivot_img_size, x-pivot_img_size:x] = display_img
                            cv2.putText(img, label, (x - pivot_img_size, y+h-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (67,67,67), 1)
                            
                            #connect face and text
                            cv2.line(img,(x+int(w/2), y+h), (x+int(w/2)-int(w/4), y+h+int(pivot_img_size/2)),(67,67,67),1)
                            cv2.line(img, (x+int(w/2)-int(w/4), y+h+int(pivot_img_size/2)), (x, y+h+int(pivot_img_size/2)), (67,67,67),1)
                            
                        elif y - pivot_img_size > 0 and x - pivot_img_size > 0:
                            #top left
                            img[y-pivot_img_size:y, x-pivot_img_size:x] = display_img
                            cv2.putText(img, label, (x - pivot_img_size, y+10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (67,67,67), 1)
                            
                            #connect face and text
                            cv2.line(img,(x+int(w/2), y), (x+int(w/2)-int(w/4), y-int(pivot_img_size/2)),(67,67,67),1)
                            cv2.line(img, (x+int(w/2)-int(w/4), y-int(pivot_img_size/2)), (x, y - int(pivot_img_size/2)), (67,67,67),1)
                            
                        elif x+w+pivot_img_size < resolution_x and y + h + pivot_img_size < resolution_y:
                            #bottom righ
                            img[y+h:y+h+pivot_img_size, x+w:x+w+pivot_img_size] = display_img
                            cv2.putText(img, label, (x+w, y+h-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (67,67,67), 1)
                            
                            #connect face and text
                            cv2.line(img,(x+int(w/2), y+h), (x+int(w/2)+int(w/4), y+h+int(pivot_img_size/2)),(67,67,67),1)
                            cv2.line(img, (x+int(w/2)+int(w/4), y+h+int(pivot_img_size/2)), (x+w, y+h+int(pivot_img_size/2)), (67,67,67),1)
                        
                    except Exception as e:
                        print("exception occured: ", str(e))
                
        cv2.imshow('img',img)
    
    if cv2.waitKey(1) & 0xFF == ord('q'): #press q to quit
        break
    
#kill open cv things        
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 374ms/step
0.9849545
0.89767855
0.98305994
detected:  Flayson Santos ( 0.89767855 )
1/1 [==============================] - 0s 311ms/step
0.9187654
0.88787204
0.9197352
detected:  Flayson Santos ( 0.88787204 )
1/1 [==============================] - 0s 366ms/step
0.93297845
0.8052308
0.8903655
detected:  Flayson Santos ( 0.8052308 )
1/1 [==============================] - 0s 393ms/step
0.8734087
0.7821005
0.82469434
detected:  Flayson Santos ( 0.7821005 )
1/1 [==============================] - 0s 358ms/step
0.92055935
0.78465146
0.88149256
detected:  Flayson Santos ( 0.78465146 )
1/1 [==============================] - 0s 307ms/step
0.8747657
0.7747011
0.84028345
detected:  Flayson Santos ( 0.7747011 )
1/1 [==============================] - 0s 312ms/step
0.92566997
0.818605
0.8779757
detected:  Flayson Santos ( 0.818605 )
1/1 [==============================] - 0s 308ms/step
0.94579184
0.8327019
0.90442675
detected:  Flayson Santos ( 0.8327019 )
1/1

1/1 [==============================] - 0s 387ms/step
0.97152054
0.87666994
0.9635755
detected:  Flayson Santos ( 0.87666994 )
1/1 [==============================] - 0s 367ms/step
0.87291586
0.8449556
0.8655888
detected:  Flayson Santos ( 0.8449556 )
1/1 [==============================] - 0s 397ms/step
0.9597455
0.8850487
0.9703835
detected:  Flayson Santos ( 0.8850487 )
1/1 [==============================] - 0s 414ms/step
0.8158293
0.8422024
0.79069906
detected:  linus  trovalds ( 0.79069906 )
1/1 [==============================] - 0s 287ms/step
0.75763583
0.82443756
0.77157116
detected:  Elon Musk ( 0.75763583 )
1/1 [==============================] - 0s 312ms/step
0.9576409
0.8961097
0.9423036
detected:  Flayson Santos ( 0.8961097 )
1/1 [==============================] - 0s 345ms/step
0.96143156
0.88097036
0.947404
detected:  Flayson Santos ( 0.88097036 )
1/1 [==============================] - 0s 376ms/step
0.96095985
0.8927647
0.98381144
detected:  Flayson Santos ( 0.8927647 )
1/1 [=